# Biostat M280 Homework 5

**Due June 15 @ 11:59PM**

Consider again the MLE of the Dirichlet-multinomial model. In [HW4](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw4/hw04.html), we worked out a Newton's method. In this homework, we explore the MM and EM approach.

## Q1

Show that, given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, the log-likelihood can be written as
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i}
+\sum_{i=1}^n \sum_{j=1}^d \sum_{k=0}^{x_{ij}-1} \ln(\alpha_j+k) - \sum_{i=1}^n \sum_{k=0}^{|\mathbf{x}_i|-1} \ln(|\alpha|+k).
$$
Hint: $\Gamma(a + k) / \Gamma(a) = a (a + 1) \cdots (a+k-1)$.

**From hw4, the log-likelihood for iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$ is **
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
With $\Gamma(a + k) / \Gamma(a) = a (a + 1) \cdots (a+k-1)$.

$$
\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j) = \alpha_j (\alpha_j + 1)\dots(\alpha_j + x_{ij}-1)
$$

$$
\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|) = |\alpha|(|\alpha| + 1) \dots (|\alpha| + |\mathbf{x}_i| - 1)
$$

So we have 
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i}
+\sum_{i=1}^n \sum_{j=1}^d \sum_{k=0}^{x_{ij}-1} \ln(\alpha_j+k) - \sum_{i=1}^n \sum_{k=0}^{|\mathbf{x}_i|-1} \ln(|\alpha|+k).
$$

## Q2

Suppose $(P_1,\ldots,P_d) \in \Delta_d = \{\mathbf{p}: p_i \ge 0, \sum_i p_i = 1\}$ follows a Dirichlet distribution with parameter $\alpha = (\alpha_1,\ldots,\alpha_d)$. Show that
$$
	\mathbf{E}(\ln P_j) = \Psi(\alpha_j) - \Psi(|\alpha|),
$$
where $\Psi(z) = \Gamma'(z)/\Gamma(z)$ is the digamma function and $|\alpha| = \sum_{j=1}^d \alpha_j$. Hint: Differentiate the identity 
$$
1 = \int_{\Delta_d} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1} \, d\mathbf{p}.
$$

## Q3

The admixture representation of the Dirichlet-multinomial distribution suggests that we can treat the unobserved multinomial parameters $\mathbf{p}_1,\ldots,\mathbf{p}_n$ as missing data and derive an EM algorithm. Show that the Q function is
$$
    Q(\alpha|\alpha^{(t)}) = 
\sum_{j=1}^d \sum_{i=1}^n \alpha_j \left[\Psi(x_{ij}+\alpha_j^{(t)}) - \Psi(|\mathbf{x}_i|+|\alpha^{(t)}|)\right] - n \sum_{j=1}^d \ln \Gamma(\alpha_j) + n \ln \Gamma(|\alpha|) + c^{(t)},
$$
where $c^{(t)}$ is a constant irrelevant to optimization. Comment on why it is not easy to maximize the Q function.

## Q4

We derive an MM algorithm for maximing $L$. Consider the formulation of the log-likelihood that contains terms $\ln (\alpha_j + k)$ and $- \ln (|\alpha|+k)$. Applying Jensen's inequality to the concave term $\ln (\alpha_j + k)$ and supporting hyperplane inequality to the convex term $- \ln (|\alpha|+k)$, show that a minorizing function to $L(\alpha)$ is
$$
	g(\alpha|\alpha^{(t)}) = - \sum_{k=0}^{\max_i |\mathbf{x}_i|-1} \frac{r_k}{|\alpha^{(t)}|+k} |\alpha| + \sum_{j=1}^d \sum_{k=0}^{\max_i x_{ij}-1} \frac{s_{jk} \alpha_j^{(t)}}{\alpha_j^{(t)}+k} \ln \alpha_j + c^{(t)},
$$
where $s_{jk} = \sum_{i=1}^n 1_{\{x_{ij} > k\}}$, $r_k = \sum_{i=1}^n 1_{\{|\mathbf{x}_i| > k\}}$, and  $c^{(t)}$ is a constant irrelevant to optimization. Maximizing the surrogate function $g(\alpha|\alpha^{(t)})$ is trivial since $\alpha_j$ are separated. Show that the MM updates are
$$
	\alpha_j^{(t+1)} = \frac{\sum_{k=0}^{\max_i x_{ij}-1} \frac{s_{jk}}{\alpha_j^{(t)}+k}}{\sum_{k=0}^{\max_i |\mathbf{x}_i|-1} \frac{r_k}{|\alpha^{(t)}|+k}} \alpha_j^{(t)}, \quad j=1,\ldots,d.
$$
The quantities $s_{jk}$, $r_k$, $\max_i x_{ij}$ and $\max_i |\mathbf{x}_i|$ only depend on data and can be pre-computed. Comment on whether the MM updates respect the parameter constraint $\alpha_j>0$.

## Q5

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using MM algorithm.

In [1]:
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    
    logpdf = 0.0
    
    t = 0.0
    for i in 1:length(x)

        if α[i] == 0 && x[i] > 0
            return - Inf
        elseif α[i] > 0
            t += lgamma(α[i] + x[i]) - lgamma(α[i]) - lfact(x[i])
        end
    end

    sumx = sum(x)
    sumα = sum(α)
    
    if sumα == 0 && sumx == 0
        return 0.0
    elseif sumα > 0 && sumx == 0 
        return - Inf
    else 
        logpdf = t + lfact(sumx) + lgamma(sumα) - lgamma(sumx + sumα)
    end
    
    return logpdf
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
    return sum(r)
end

dirmult_logpdf

In [4]:
#Pkg.add("SpecialFunctions")
using SpecialFunctions

function dirmult_score(x::Vector, α::Vector)
    
    score = zeros(length(α))
    sumx = sum(x)
    sumα = sum(α)
    
    for j in 1:length(x)
        #println("length(x) is", length(x))
        score[j] = digamma(α[j] + x[j]) - digamma(α[j]) 
    end
    
    common_term = digamma(sumx + sumα) - digamma(sumα)

    score .= score - common_term
    return score
end

function dirmult_score!(s::Vector, X::Matrix, α::Vector)
    d, n = size(X)
    for i in 1:d
        for j in 1:n
            s[i] += dirmult_score(X[:, j], α)[i]
        end 
    end
    return s
end


function dirmult_score(X::Matrix, α::Vector)
    s = zeros(size(X, 1))
    dirmult_score!(s, X, α)
    return s
end

#dirmult_score(X, α)

dirmult_score (generic function with 2 methods)

In [8]:
"""
    dirmult_newton(X)

Find the MLE of Dirichlet-multinomial distribution using Newton's method.

# Argument
* `X`: an `n`-by-`d` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: a `d` vector of starting point (optional). 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
* `maximum`: the log-likelihood at MLE.   
* `estimate`: the MLE. 
* `gradient`: the gradient at MLE. 
* `hessian`: the Hessian at MLE. 
* `se`: a `d` vector of standard errors. 
* `iterations`: the number of iterations performed.
"""
function dirmult_newton(X::Matrix;
            printout = false,
            maxiters::Int = 100, tolfun::Float64 = 1e-6,
            α0::Vector{Float64} = dirmult_startingvalue_mom(X))
    
    rowindex = find(sum(X, 2))
    colind = find(sum(X, 1))
    X = X[rowindex, colind] 
     
    d, n = size(X)

    
    # set default starting point from Q7
    α = α0[rowindex]
    αnew = similar(α)
    
    # preallocation: score, obsinfo diagonals & constant, 
    ∇ = zeros(d)
    diagonals = zeros(d)
    invdiagonals = zeros(d)
    
    outerproduct = zeros(d, d)
    newton_direction = zeros(d)
    
    constantC = 0.0
    invconstantC = 0.0
    loglike_new = 0.0
    

    loglike_old = dirmult_logpdf(X, α)
    
    # Newton loop
    num_iter = 0
    for iter in 1:maxiters
        
        ###############evaluate gradient (score)
        dirmult_score!(∇, X, α) #64 element array 
        
        # compute Newton's direction
        ############### first get the P.D. approximation for obs info
        ############### obs info
        diagoals, constantC = dirmult_obsinfo!(diagonals, X, α)
        invdiagonals .= 1.0 ./ diagoals
        invconstantC = 1.0 / constantC
        
        tmp = sum(invdiagonals)
        if invconstantC <= tmp
            invconstantC = 1.01 * tmp
            print("reset constant C")
        end
        
        # now A = D - c 11'
        #the newton direction = A^(-1) * score 
        
        newton_direction .= invdiagonals .* ∇ .+ 
                A_mul_Bt!(outerproduct, invdiagonals, invdiagonals) * 
                ∇ /
                (invconstantC - tmp)

        # line search loop
        step = 1.0
        
        # constrain alpha to have all postive elements 
        for i in eachindex(α)       
            if newton_direction[i] < 0
                step = min(- α[i] / newton_direction[i] * 0.99, step)
            end
        end
        
        for lsiter in 1:10
            
            αnew .= α .+ step .* newton_direction
            loglike_new = dirmult_logpdf(X, αnew)
         
            if loglike_new > loglike_old
                break
            end
            
            if lsiter < 10
                step /= 2.0
            end
        end
        
        copy!(α, αnew)
        
        if printout
            println("iteration ", iter, ", loglike = ", loglike_new)
        end
        
        # check convergence criterion
        if abs(loglike_new - loglike_old) < tolfun * (abs(loglike_old) + 1)
            break;
        end
        
        loglike_old = loglike_new
        num_iter += 1  
    end
    
        lastα = zeros(length(α0))
        lastα[rowindex] = α
    
        last∇ = zeros(length(α0))
        last∇[rowindex] = dirmult_score(X, α)
    
        obsinfo = zeros(length(α0), length(α0))
        diags, c = dirmult_obsinfo!(diagonals, X, α) 
        obsinfo[rowindex, rowindex] = Diagonal(diags) - c
    
        return loglike_new, num_iter, lastα, last∇, obsinfo
end
#trainset = readcsv("optdigits.tra", Int) 
#X = trainset[trainset[:, end] .== 0, 1:64]
#X = X'
#@time dirmult_newton(X; printout = true,tolfun=1e-8,maxiters = 100)

dirmult_newton

In [6]:
#Pkg.add("SpecialFunctions")
function dirmult_startingvalue_mom(X::Matrix)
    
    d, n = size(X)
    αsum_ini = ones(64)
    
    vec = zeros(n)
    k = 0.0
    
    for j in 1:d
        
        vec = X[j,:]'./ sum(X, 1)
        num = sum(vec.^2)
        denum = sum(vec)
    
        if denum > 0
            k += num / denum
        end
    end

    αsum_ini = (d - k) / (k - 1.0)

    grandsum = sum(X)
    
    α_ini = zeros(Float64, d)
    
    for i in 1:d
        α_ini[i] = αsum_ini * sum(X, 2)[i] / grandsum
    end

    return α_ini
end

#α0 = dirmult_startingvalue_mom(X)




dirmult_startingvalue_mom (generic function with 1 method)

In [1]:
"""
    dirmult_mm(X)

Find the MLE of Dirichlet-multinomial distribution using MM algorithm.

# Argument
* `X`: an `d`-by-`n` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: starting point. 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
# Output
* `logl`: the log-likelihood at MLE.   
* `niter`: the number of iterations performed.
# `α`: the MLE.
* `∇`: the gradient at MLE. 
* `obsinfo`: the observed information matrix at MLE. 
"""
function dirmult_mm(
    X::AbstractMatrix; 
    α0::Vector = dirmult_mom(X), 
    maxiters::Int = 100, 
    tolfun = 1e-6
    )
    
    # your code here
    
    # for output
    
end

dirmult_mm

## Q6

Re-do [HW4 Q9](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw4/hw04.html#Q9) using your new `dirmult_mm` function. Compare the number of iterations and run time by MM algorithm to those by Newton's method. Comment on the efficiency of Newton's algorithm vs MM algorithm for this problem.

## Q7

Finally let us re-consider the EM algorithm. The difficulty with the M step in EM algorithm can be remedied. Discuss how we can further minorize the $\ln \Gamma(|\alpha|)$ term in the $Q$ function to produce a minorizing function with all $\alpha_j$ separated. For this homework, you do **not** need to implement this EM-MM hybrid algorithm. Hint: $z \mapsto \ln \Gamma(z)$ is a convex function.